In [5]:
import time
import os
import cv2
import pandas as pd
import torch
from ultralytics import YOLO
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.transforms import functional as F

# Define Image Directory (folder named 'images' and put the 10 images inside)
IMG_DIR = 'images'
image_files = [os.path.join(IMG_DIR, f) for f in os.listdir(IMG_DIR) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Model 1: YOLOv8 (Nano version for speed)
yolo_model = YOLO('yolov8n.pt')

# Model 2: Faster R-CNN (Pre-trained)
weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
frcnn_model = fasterrcnn_resnet50_fpn_v2(weights=weights)
frcnn_model.eval()

# Threshold for detection confidence
CONF_THRESHOLD = 0.5

results_data = []

print(f"Processing {len(image_files)} images...")

# Loop for each image
for img_path in image_files:
    img_name = os.path.basename(img_path)

    # Load Image
    cv_img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)

    # YOLOv8
    start_time = time.time()
    yolo_results = yolo_model(img_path, verbose=False)[0] # Run inference
    yolo_time = time.time() - start_time

    # Extract YOLO stats
    yolo_boxes = yolo_results.boxes
    yolo_count = len(yolo_boxes)
    yolo_avg_conf = yolo_boxes.conf.mean().item() if yolo_count > 0 else 0.0

    # Faster R-CNN
    # Transform image for PyTorch
    img_tensor = F.to_tensor(img_rgb).unsqueeze(0)

    start_time = time.time()
    with torch.no_grad():
        predictions = frcnn_model(img_tensor)[0]
    frcnn_time = time.time() - start_time

    # Filter Faster R-CNN results by confidence score
    high_conf_indices = [i for i, score in enumerate(predictions['scores']) if score > CONF_THRESHOLD]
    frcnn_count = len(high_conf_indices)
    frcnn_avg_conf = predictions['scores'][high_conf_indices].mean().item() if frcnn_count > 0 else 0.0

    # Using Canny Edge Detection to calculate "Edge Density" (how 'busy' the image is)
    edges = cv2.Canny(cv_img, 100, 200)
    edge_density = (edges > 0).mean() # Percentage of pixels that are edges

    # Append to data
    results_data.append({
        "Image": img_name,
        "YOLO Time (s)": round(yolo_time, 4),
        "YOLO Objects": yolo_count,
        "YOLO Avg Prob": round(yolo_avg_conf, 2),
        "FRCNN Time (s)": round(frcnn_time, 4),
        "FRCNN Objects": frcnn_count,
        "FRCNN Avg Prob": round(frcnn_avg_conf, 2),
        "Edge Density": round(edge_density, 4)
    })

df = pd.DataFrame(results_data)
print(df)

# Save to CSV
#df.to_csv("comparison_results.csv", index=False)

Processing 10 images...
                                               Image  YOLO Time (s)  \
0          Belmont Park In San Diego, California.jpg         0.3887   
1             Chicano Park graffiti painting art.jpg         0.2188   
2         flower-fields-san-diego-carlsbad-ranch.jpg         0.2308   
3             scripps-pier-san-diego-photogrpahy.jpg         0.2148   
4  centenial park coronado night view down town s...         0.1940   
5                                 SR-163-closure.jpg         0.2505   
6                      sunset-cliffs-san-diego-2.jpg         0.2397   
7              coronado-view-down-town-san-diego.jpg         0.1988   
8                     balboa-park-pond-san-diego.jpg         0.2007   
9                                  san-diego-zoo.jpg         0.1731   

   YOLO Objects  YOLO Avg Prob  FRCNN Time (s)  FRCNN Objects  FRCNN Avg Prob  \
0             4           0.35         17.6125             53            0.72   
1             1           0.49  